In [ ]:
import json
import pandas as pd
import numpy as np
import calendar, datetime
import datetime

file = 'wemeal-export.json'
user_file = 'wemeal-export-users.json'

In [ ]:
#List of users. 
count=0
user_list = []

for i in all_users:
    firstName =  all_users[i]["firstName"]
    lastName = all_users[i]["lastName"]
    
    

    try:
    
        created = all_users[i]["Created_at"]
        email = all_users[i]["email"]
        
    except:
    
        email = " N/A"

    if created > 1368195200:
        count=count+1
        print "%s, %s, %s" % (lastName, firstName, email)
        user_list.append([lastName, firstName, email])
df= pd.DataFrame(user_list)
df.columns= ['LastName', 'FirstName', 'Email']

print "The total count is %s" % count
df


In [ ]:
# Who's created meals. 
meal_table = config['meal']
meal_items = config['meal_items']

meal_list = []
for i in meal_table:
    user_id =  meal_table[i]['meal_createdby']
    # print user_id
    if user_id in user_table.keys():
        # print "HI"
        user_info = user_table[user_id]
        #print user_info
        firstName =  user_info["firstName"]
        lastName = user_info["lastName"]
        meal_created = dt(meal_table[i]["meal_date"])
        potluck = meal_table[i]["meal_pre_everyonebringfood"]
        
        #print "%s, %s, isPotluck: %s" % (firstName, lastName, potluck)
        meal_list.append([firstName, lastName, potluck,meal_created])
df= pd.DataFrame(meal_list)
df.columns= ['FirstName', 'LastName', 'isPotluck?','date']
df.sort("date")

In [ ]:
def get_date(meal_id):
    return dt(meal_table[meal_id]["meal_date"])

def get_address(meal_id):
    return meal_table[meal_id]["meal_location"]

In [ ]:
#Who's bringing what? 

meal_items = config['meal_items']
#given meal:

potluck_meal_items = []

for i in meal_table:
    potluck = meal_table[i]["meal_pre_everyonebringfood"]
    host = get_name(meal_table[i]['meal_createdby'])
    date = get_date(i)
    address = get_address(i)
    if potluck == "YES":
        try: 
            list_of_items = meal_items[i]
        except:
            "Skipped "+i
        
        for j in list_of_items:
            food = j['name']
            if j['user_id'] == '' or j['user_id'] == "":
                firstName = "null"
                lastName ="null"
            else: 
                user_info = user_table[j['user_id']]
                #print user_info
                firstName =  user_info["firstName"]
                lastName = user_info["lastName"]
            if firstName == "null":
                pass
            else:
                potluck_meal_items.append([firstName,lastName,food,host,date,address]) 
        

In [ ]:
def get_dateformat(timestamp):
    return datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
config_users
df = pd.DataFrame(columns=['email','firstname','lastname','created_at'])
d = []
for i in config_users:
    user = config_users[i]
    #print user
    try:
        email=user['email']
    except:
        email="unknown"
    
    firstname = user['firstName']
    lastname = user['lastName']
    try:
        created_at = user['Created_at']
    except:
        created_at = -1
    d.append([email,firstname,lastname,created_at,get_date(created_at)])



In [ ]:

df= pd.DataFrame(d,columns=["Email","FirstName","LastName","Create_at","created_at_date"])

In [ ]:
df= pd.DataFrame(d,columns=["Email","FirstName","LastName","Create_at","created_at_date"])
df.sort('Create_at',ascending=-1)

In [ ]:

timestamp = 1455555555

In [ ]:
df[c('email')]

In [ ]:
df1= df.loc[250:,["Email","FirstName","LastName"]]

In [ ]:
# Read the JSON .

all_user_file = "wemeal-users-aug5.json"

#config = json.loads(open(file).read())
config_and_users = json.loads(open(all_user_file).read())

In [ ]:
print config_and_users


In [ ]:
df = pd.DataFrame(columns=['email','firstname','lastname','created_at'])
df1 = []

for i in config_and_users:
    try:
        e= config_and_users[i]['email']
    except:
        e="none"
    try: 
        f= config_and_users[i]['firstName']
    except:
        f= "none"
    try: 
        l=config_and_users[i]['lastName']
    except:
        l="none"
    try: 
        d=config_and_users[i]['Created_at']
    except:
        d= 0
        
    df1.append([e,f,l,d,get_dateformat(d)])

df= pd.DataFrame(df1,columns=["Email","FirstName","LastName","Create_at","Date"])
df.sort('Create_at',ascending=-1)
    
df.to_csv('out-android.csv', sep='\t', encoding='utf-8')

In [ ]:
import csv
import json
import pandas as pd
import numpy as np
import calendar, datetime
import datetime

def dt(u): return datetime.datetime.utcfromtimestamp(u)


In [ ]:
def get_meal_location(meal_id):
    x = config_meal_data[meal_id]['meal_location']
    if config_meal_data[meal_id]['meal_apprtment'] != "":
        x=x+" Apt: %s"%config_meal_data[meal_id]['meal_apprtment']
    return x

def basic_meal_data(config_meal_data, date=1470342643):
    df = get_data_after_date(config_meal_data,date)
    return df.sort('total_attendees',ascending=-1)

def get_data_after_date(config_meal_data,date):
    df1 = []
    new_list = []
    for i in config_meal_data:
        meal_data = config_meal_data[i]
        meal_id=i
        m_date= meal_data['meal_date']
        total_attendees= meal_data['total_attendees']
        host_name = get_name(config_meal_data[i]['meal_createdby'])
        
        if m_date>date:
            #print "%s , %s, Total: %s" %( host_name, dt(m_date),total_attendees)
            df1.append([host_name,meal_id, dt(m_date),int(total_attendees)])
    return pd.DataFrame(df1,columns=['meal_date','meal_id','host_name','total_attendees'])

def list_members_get_members_of_meal(meal_id):
    try:
        all_meal_info = config_meal_members[meal_id]
    except:
        all_meal_info = "empty"
    return all_meal_info

def get_food_item(config_meal_item,meal_id,user_id):
    food_items_list = []
    try: 
        x= config_meal_item[meal_id]
        for i in x:
            if i['user_id']==user_id:
                food_items_list.append(i['name'])
    except:
        "no items"
    #print food_items_list
    return mk_list_str(food_items_list)

def mk_list_str(food_items_list):
    if len(food_items_list)==0:
        return ""
    elif len(food_items_list)==1:
        
        return food_items_list[0]
    else:
        termp=food_items_list[0]
        "Sup"
        for i in range(len(food_items_list)-2):
           termp = termp +", "+food_items_list[i+1]
        termp = termp +", and " +food_items_list[-1]
        return termp

def get_name(user_id):
    try: 
        user_info = config_user_data[user_id]
        firstName =  user_info["firstName"]
        lastName = user_info["lastName"]
    except:
        firstName = "No"
        lastName = "User"

    return "%s %s" %(firstName,lastName)

def get_email_name2(user_id):
    try: 
        user_info = config_user_data[user_id]
        firstName =  user_info["firstName"]
        lastName = user_info["lastName"]
        email = user_info["email"]
    except:
        firstName = "No"
        lastName = "User"
        email = ""
    return (firstName,lastName, email)
    
def list_of_food_items(meals_this_week):
    listitems=[]
    for i in range(len(meals_this_week)):
        meal_id = meals_this_week['meal_id'][i]
        print meal_id
        x = list_members_get_members_of_meal(meal_id)
        for i in range(len(x)):
            
            #For each user: 
            meal_id=meal_id
            meal_time=dt(config_meal_data[meal_id]['meal_date'])
            meal_location=get_meal_location(meal_id)
            try: 
                user_id= x[i]['user_id']
            except:
                user_id= "none"
            x_l = config_meal_data[meal_id]['meal_createdby']
            host_name = get_name(x_l)
            name_email=get_email_name2(user_id)
            first=name_email[0]
            last=name_email[1]
            email=name_email[2]
            food_item=get_food_item(config_meal_item,meal_id,user_id)
            listitems.append([meal_id,meal_time,meal_location,user_id,host_name,first,last,email,food_item])
        # Add what each person is bringing. 
    return pd.DataFrame(listitems,columns=["meal_id","meal_time","host_name","meal_location","user-id","name","last","email","food_item"])

    
    

In [ ]:
# Read the JSON .

#config = json.loads(open(file).read())
config_meal_data = json.loads(open("data/wemeal-export-meal.json").read())
config_user_data = json.loads(open("data/wemeal-export-user.json").read())
config_meal_item = json.loads(open("data/wemeal-export-mealitems.json").read())
config_meal_members = json.loads(open("data/wemeal-export-mealmembers.json").read())

def write_csv_of_meal_info(date=1470342643):
    # This computes Meals happening this week and how many people are going.
    
    meals_this_week = basic_meal_data(config_meal_data)
    meals_this_week.to_csv('meals_aug_4.csv', sep='\t', encoding='utf-8')
    df = list_of_food_items(meals_this_week)
    df.to_csv('list_of_food_items_aug_4.csv', sep=',', encoding='utf-8')
    
write_csv_of_meal_info()

In [ ]:
#Get all users info in csv file format

def output_user_info():
    #
    #get user info == 
    df1 = []
    print len(config_user_data)
    for i in config_user_data:
        print i
        try:
            e= config_user_data[i]['email']
        except:
            e="none"
        try: 
            f= config_user_data[i]['firstName']
        except:
            f= "none"
        try: 
            l=config_user_data[i]['lastName']
        except:
            l="none"
        try: 
            d=config_user_data[i]['Created_at']
        except:
            d= 0
        #print [e,f,l,d,get_dateformat(d)]
        df1.append([e,f,l,d])
    

    df= pd.DataFrame(df1,columns=["Email","FirstName","LastName","Create_at"])
    df.sort('Create_at',ascending=-1)
    print "start"
    df.to_csv('user_data_out_aug10.csv', sep=',', encoding='utf-8')
    print "done"

In [ ]:
output_user_info()